In [ ]:
'''poker model control script'''

# pylint: disable=E1101, E1601, W0612

import gc
import mysql.connector

import funcs_db
import funcs_poker

import pdeck

In [ ]:
gc.collect()

# OPEN CONNECTION TO POKER DB

In [ ]:
poker_db = mysql.connector.connect(user='root', host='127.0.0.1', database='poker')

# GAME PARAMETERS

In [ ]:
number_of_players = 6
player0_stack = 1000
player1_stack = 1000
player2_stack = 1000
player3_stack = 1000
player4_stack = 1000
player5_stack = 1000
small_blind = 10
big_blind = 2 * small_blind

funcs_db.sql_delete_all(table='games', poker_db=poker_db)
funcs_db.sql_delete_all(table='history', poker_db=poker_db)
funcs_db.sql_delete_all(table='decision_points', poker_db=poker_db)
funcs_db.sql_delete_all(table='possible_moves', poker_db=poker_db)
funcs_db.sql_insert_games(number_of_players=number_of_players, player0_stack=player0_stack, \
                          player1_stack=player1_stack, player2_stack=player2_stack, player3_stack=player3_stack, \
                          player4_stack=player4_stack, player5_stack=player5_stack, small_blind=small_blind, \
                          big_blind=big_blind, poker_db=poker_db)

# TABLE INITIALIZATION

In [ ]:
TABLE = funcs_poker.table_init(number_of_players=number_of_players, player0_stack=player0_stack, \
                               player1_stack=player1_stack, player2_stack=player2_stack, player3_stack=player3_stack, \
                               player4_stack=player4_stack, player5_stack=player5_stack)
POT = TABLE[0]
PLAYERS = TABLE[1]

# GAME INITIALIZATION

## shift the players position by 1

In [ ]:
for player in PLAYERS:

    player.add_position((player.position_nr() + 1) % len(PLAYERS))

## create deck and board

In [ ]:
DECK = pdeck.Deck()
BOARD = DECK.make_board()

## create hands and add them to players

In [ ]:
HANDS = [DECK.make_hand() for player in PLAYERS]
for i in range(len(PLAYERS)):

    PLAYERS[i].add_hand(HANDS[i])

## define players order

In [ ]:
PLAYERS_ORDERED = {player.position_nr(): player for player in PLAYERS}
PLAYERS_ORDERED = dict(sorted(PLAYERS_ORDERED.items()))

## define players move flags

In [ ]:
PLAYER_MOVE_FLAGS = [1 for player in PLAYERS]

# PREFLOP PHASE

## small blind and big blind moves

In [ ]:
funcs_poker.move(poker_db=poker_db, player=PLAYERS_ORDERED[0], pot=POT, move='small_blind', amount=small_blind)
funcs_poker.move(poker_db=poker_db, player=PLAYERS_ORDERED[1], pot=POT, move='big_blind', nr=1, amount=big_blind)

## further moves

In [ ]:
phase = 0
nr = 2
for i in range(2, 6):
    
    move, amount = funcs_db.decision_point(poker_db=poker_db, player_name=PLAYERS_ORDERED[i].general_name(), \
                                           hand=PLAYERS_ORDERED[i].player_hand_simple(), stack=PLAYERS_ORDERED[i].stack(), \
                                           pot=POT.show_pot(), position=PLAYERS_ORDERED[i].position_nr(), phase=phase, \
                                           nr=nr)
    funcs_poker.move(poker_db=poker_db, player=PLAYERS_ORDERED[i], pot=POT, move=move, phase=phase, nr=nr, amount=amount)
    
    # save move information for every players

    nr = nr + 1

## while sum(PLAYER_MOVE_FLAGS) != 0: # extend with other conditions

# FLOP PHASE

# TURN PHASE

# RIVER PHASE

# CLOSE CONNECTION

In [ ]:
poker_db.close()